In [6]:
from math import exp, log

import torch
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.special import expit

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression,  LogisticRegressionCV, LinearRegression, Ridge, ElasticNet
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve, CalibrationDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LearningCurveDisplay, ShuffleSplit, learning_curve, validation_curve
from sklearn.model_selection import train_test_split
from numpy import linalg as LA

import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import limix
import pdb

from numpy import array
from numpy_sugar.linalg import economic_qs_linear
from glimix_core.lmm import LMM


%matplotlib inline 

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams['figure.figsize'] = 10, 8

np.seterr(divide='ignore', invalid='ignore')
np.random.seed(42)


In [7]:
def ece(y_true, y_pred, n_bins=10, title='text'):
    x_axis = np.arange(0, 1.1, (1.0)/n_bins)
    y_axis = np.zeros(x_axis.shape)
    x_axis_new = np.zeros(x_axis.shape)
    bin_count = np.zeros(x_axis.shape)
    zero_indices = []
    N = len(y_true)
    score = 0
    for i, x in enumerate(x_axis):
        if(i==0):
            continue
        bin_outputs = y_true[np.logical_and(y_pred<=x, y_pred>x_axis[i-1])]
        bin_preds = y_pred[np.logical_and(y_pred<=x, y_pred>x_axis[i-1])]
        
            
        if(len(bin_outputs)>0):
            y_axis[i]=bin_outputs.mean()
            avg_pred = bin_preds.mean()
            x_axis_new[i] = avg_pred
        else:
            y_axis[i]=0
            avg_pred = 0
            x_axis_new[i] = x
            zero_indices += [i] 
        
        
        bin_count[i] = len(bin_outputs)
        score+=abs(y_axis[i]-avg_pred)*((1.0*len(bin_outputs))/N)
    return score  

In [8]:
run_baselines=True

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline

names = [
    "Neural Net",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bayes",
]

classifiers = [
    MLPClassifier(alpha=1, max_iter=10000, early_stopping=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10),
    AdaBoostClassifier(),
    GaussianNB(),
]
datasets = [
    'bn_sim100_',
    'sp0_3_sim100_',
    'sp0_5_sim100_',
    'tgp_sim100_',
    'hgdp_sim100_',
]


EXP_REPS = 10
NUM_SNPS = 100
epsilon = 1e-2


# Results
naive_est_marginal_betas = torch.zeros((EXP_REPS, NUM_SNPS))
plain_iptw_est_marginal_betas = torch.zeros((EXP_REPS, NUM_SNPS))
calib_iptw_est_marginal_betas = torch.zeros((EXP_REPS, NUM_SNPS))
plain_aipw_est_marginal_betas = torch.zeros((EXP_REPS, NUM_SNPS))
calib_aipw_est_marginal_betas = torch.zeros((EXP_REPS, NUM_SNPS))

beta_pca = torch.zeros((EXP_REPS, NUM_SNPS))
beta_fa = torch.zeros((EXP_REPS, NUM_SNPS))
beta_lmm = torch.zeros((EXP_REPS, NUM_SNPS))
beta_true = np.zeros((EXP_REPS, NUM_SNPS))
cal_scores = torch.zeros((EXP_REPS, NUM_SNPS, 2))
cal_scores_train = torch.zeros((EXP_REPS, NUM_SNPS, 2))


for CLASSIFIER_ID, classifier in enumerate(classifiers):
    for dataset in datasets:
        for dataset_idx in range(0,EXP_REPS):
            print("Working with Dataset "+str(dataset_idx))
            
            simulated_data = torch.load('simulated_gwas/'+dataset+str(dataset_idx)+'.pt')

            N = simulated_data['snps'].shape[1]
            
            M = simulated_data['snps'].shape[0]
            G = simulated_data['snps'][:int(0.8*M)]
            Y = simulated_data['outcomes'][:int(0.8*M)]
            Z = simulated_data['clusters'][:int(0.8*M)]

            G_test = simulated_data['snps'][int(0.8*M):]
            Y_test = simulated_data['outcomes'][int(0.8*M):]
            Z_test = simulated_data['clusters'][int(0.8*M):]
            beta_true[dataset_idx] = betas = simulated_data['true_betas']



            # iterate through snp vector sequentially



            for snp_index in range((G.shape[1])):


                print("Reading SNP "+str(snp_index))


                T = G[:, snp_index]
                T_test = G_test[:, snp_index]

                X = np.concatenate((G[:, :snp_index], G[:, snp_index+1:]), axis=1)

                X_test = np.concatenate((G_test[:, :snp_index], G_test[:, snp_index+1:]), axis=1)

       
                if(run_baselines):
                    pc = PCA(n_components=10)
                    pc_snps = StandardScaler().fit_transform(pc.fit_transform(X))

                    G_pca = np.concatenate((T.reshape(-1,1), pc_snps), axis=1)
                    beta_pca[dataset_idx][snp_index] = (Ridge().fit(G_pca, Y).coef_)[0]

                    pc_snps_test = StandardScaler().fit_transform(pc.transform(X_test))
                    G_pca = np.concatenate((T_test.reshape(-1,1), pc_snps_test), axis=1)
                    

                    QS = economic_qs_linear(X)
                    covariates = X
                    y = Y
                    lmm = LMM(y, T, QS)
                    lmm.fit(verbose=False)
                    beta_lmm[dataset_idx][snp_index] = lmm.beta[0]

                    fa = FactorAnalysis(n_components=10)
                    fa_snps = fa.fit_transform(X)
                    fa_snps = StandardScaler().fit_transform(fa_snps)
                    G_fa = np.concatenate((T.reshape(-1,1), fa_snps), axis=1)
                    beta_fa[dataset_idx][snp_index] = (Ridge().fit(G_fa, Y).coef_)[0]

                    fa_snps_test = StandardScaler().fit_transform(fa.transform(X_test))
                    G_fa = np.concatenate((T_test.reshape(-1,1), fa_snps_test), axis=1)
                    

                reg1 = Ridge().fit(X[T==1], Y[T==1])
                reg0 = Ridge().fit(X[T==0], Y[T==0])


                naive_est_marginal_betas[dataset_idx][snp_index] = (reg1.predict(X) - reg0.predict(X)).mean() # assuming linear model
  
                prop_model = classifiers[CLASSIFIER_ID].fit(X, T)


                prop_scores = np.clip(prop_model.predict_proba(X)[:, 1], a_min = epsilon, a_max = 1-epsilon) # clipping necessary before inverting
                prop_scores_test = np.clip(prop_model.predict_proba(X_test)[:, 1], a_min = epsilon, a_max = 1-epsilon)



                cal_scores[dataset_idx][snp_index][0] = ece(T_test, prop_model.predict_proba(X_test)[:, 1], title='Before calibration [Test]')
                cal_scores_train[dataset_idx][snp_index][0] = ece(T, prop_model.predict_proba(X)[:, 1], title='Before calibration [Train]')


                plain_iptw_est_marginal_betas[dataset_idx][snp_index] = ((T/prop_scores)*Y - ((1-T)/(1-prop_scores))*Y).mean()


                plain_aipw_est_marginal_betas[dataset_idx][snp_index] = plain_iptw_est_marginal_betas[dataset_idx][snp_index] + ((prop_scores - T)*reg0.predict(X)/(1 - prop_scores) + (prop_scores - T)*reg1.predict(X)/(prop_scores)).mean()

                prop_model_calib = CalibratedClassifierCV(prop_model, method='isotonic', ensemble=True, cv='prefit') # works
                prop_model_calib.fit(X, T)


                prop_scores = np.clip(prop_model_calib.predict_proba(X)[:, 1], a_min = epsilon, a_max = 1-epsilon)

                prop_scores_test = np.clip(prop_model_calib.predict_proba(X_test)[:, 1], a_min = epsilon, a_max = 1-epsilon)

                cal_scores[dataset_idx][snp_index][1] = ece(T_test, prop_model_calib.predict_proba(X_test)[:, 1], title='After calibration [Test]')
                cal_scores_train[dataset_idx][snp_index][1] = ece(T, prop_model_calib.predict_proba(X)[:, 1], title='After calibration [Train]')
                
                calib_iptw_est_marginal_betas[dataset_idx][snp_index] = ((T/prop_scores)*Y - ((1-T)/(1-prop_scores))*Y).mean()


                calib_aipw_est_marginal_betas[dataset_idx][snp_index] = calib_iptw_est_marginal_betas[dataset_idx][snp_index] + ((prop_scores - T)*reg0.predict(X)/(1 - prop_scores) + (prop_scores - T)*reg1.predict(X)/(prop_scores)).mean()



        EXP_REPS = len(calib_iptw_est_marginal_betas)
        result_norms = torch.zeros(EXP_REPS, 12)
        for i in range(EXP_REPS):

            result_norms[i][0] = (LA.norm(calib_iptw_est_marginal_betas[i] - beta_true[i]))
            result_norms[i][1] = (LA.norm(plain_iptw_est_marginal_betas[i] - beta_true[i]))
            result_norms[i][2] = (LA.norm(calib_aipw_est_marginal_betas[i] - beta_true[i]))
            result_norms[i][3] = (LA.norm(plain_aipw_est_marginal_betas[i] - beta_true[i]))

            result_norms[i][4] = (LA.norm(beta_pca[i] - beta_true[i]))
            result_norms[i][5] = (LA.norm(beta_fa[i] - beta_true[i]))
            result_norms[i][6] = (LA.norm(beta_lmm[i] - beta_true[i]))
            result_norms[i][7] = (LA.norm(naive_est_marginal_betas[i] - beta_true[i]))
            result_norms[i][8] = (cal_scores_train[i][:, 0]).mean(axis=0)
            result_norms[i][9] = (cal_scores_train[i][:, 0]).std(axis=0)/np.sqrt(len(cal_scores[i]))
            result_norms[i][10] = (cal_scores_train[i][:, 1]).mean(axis=0)
            result_norms[i][11] = (cal_scores_train[i][:, 1]).std(axis=0)/np.sqrt(len(cal_scores[i]))
        torch.set_printoptions(precision=5, sci_mode=False)
        print("printing result")
        print(result_norms)
        print(result_norms.mean(axis=0))
        print(result_norms.std(axis=0)/np.sqrt(len(result_norms)))
        

Working with Dataset 0
Reading SNP 0
Reading SNP 1
Reading SNP 2
Reading SNP 3
Reading SNP 4
Reading SNP 5
Reading SNP 6
Reading SNP 7
Reading SNP 8
Reading SNP 9
Reading SNP 10
Reading SNP 11
Reading SNP 12
Reading SNP 13
Reading SNP 14
Reading SNP 15
Reading SNP 16
Reading SNP 17
Reading SNP 18
Reading SNP 19
Reading SNP 20
Reading SNP 21
Reading SNP 22
Reading SNP 23
Reading SNP 24
Reading SNP 25
Reading SNP 26
Reading SNP 27
Reading SNP 28
Reading SNP 29
Reading SNP 30
Reading SNP 31
Reading SNP 32
Reading SNP 33
Reading SNP 34
Reading SNP 35
Reading SNP 36
Reading SNP 37
Reading SNP 38
Reading SNP 39
Reading SNP 40
Reading SNP 41
Reading SNP 42
Reading SNP 43
Reading SNP 44
Reading SNP 45
Reading SNP 46
Reading SNP 47
Reading SNP 48
Reading SNP 49
Reading SNP 50
Reading SNP 51
Reading SNP 52
Reading SNP 53
Reading SNP 54
Reading SNP 55
Reading SNP 56
Reading SNP 57
Reading SNP 58
Reading SNP 59
Reading SNP 60
Reading SNP 61
Reading SNP 62
Reading SNP 63
Reading SNP 64
Reading SNP 